In [1]:
from itertools import product

import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import (accuracy_score, recall_score, precision_score,
                             roc_auc_score, confusion_matrix)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     train_test_split)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from scipy import stats
from statsmodels.stats.contingency_tables import mcnemar

from src.radiomics.models.data import load_data, get_formatted_data
from src.radiomics.models.utils import RemoveHighlyCorrelatedFeatures
from src.roc_comparison.compare_auc_delong_xu import delong_roc_test, delong_roc_variance


In [2]:

clinical_df = pd.read_csv("/home/valentin/python_wkspce/plc_segmentation/data/clinical_info_updated.csv").set_index("patient_id")

In [3]:
ids = clinical_df.index
ids = [
    i for i in ids if i not in
    ["PatientLC_71", "PatientLC_21", "PatientLC_63", "PatientLC_72"]
]
clinical_df = clinical_df.loc[ids, :]


In [4]:
clinical_df.shape

(105, 16)

In [5]:
def get_gridsearch():
    scaler = StandardScaler()

    clf_lr = LogisticRegression(penalty="none", solver='sag', max_iter=1000)
    clf_rf = RandomForestClassifier()

    pipe = Pipeline(steps=[
        ('normalization', scaler),
        ('feature_removal', RemoveHighlyCorrelatedFeatures()),
        ('feature_selection', None),
        ('classifier', None),
    ])

    F_OPTIONS = [1, 2, 3, 5]
    K_OPTIONS = [k for k in range(1, 11)]

    param_grid = [
        {
            'feature_selection': [SelectKBest(f_classif)],
            'feature_selection__k': F_OPTIONS,
            'classifier': [clf_rf],
            'classifier__n_estimators': [100, 150]
        },
        {
            'feature_selection': [SelectKBest(f_classif)],
            'feature_selection__k': F_OPTIONS,
            'classifier': [clf_lr],
        },
        {
            'feature_selection': [PCA()],
            'feature_selection__n_components': K_OPTIONS,
            'classifier': [clf_lr],
        },
        {
            'feature_selection': [PCA()],
            'feature_selection__n_components': K_OPTIONS,
            'classifier': [clf_rf],
        },
    ]

    return GridSearchCV(pipe,
                        param_grid,
                        cv=StratifiedKFold(),
                        n_jobs=23,
                        refit=True,
                        verbose=1,
                        scoring="roc_auc")



In [6]:
search = get_gridsearch()

In [7]:
df  = load_data(
    "../data/processed/radiomics/extracted_features.csv",
    # "../data/processed/radiomics/extracted_features_auto.csv",
    "../data/clinical_info_updated.csv",
)

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [9]:
clinical_df["Peri bronchovascular thickening"].isna().sum()

7

In [10]:
def select_id(df, plc_status=0, missclassified=True, criterion="thickening"):
    crit = {
        "thickening": "Peri bronchovascular thickening",
        "pet": "PET_lymphangitis_Visual_analysis",
    }
    if missclassified:
        return ((df["plc_status"] == plc_status)
                & (~df[crit[criterion]].isna())
                & (df.plc_status != df[crit[criterion]]) & (df.is_chuv == 1))
    else:
        return ((df["plc_status"] == plc_status)
                & (~df[crit[criterion]].isna())
                & (df.plc_status == df[crit[criterion]])
                & (df.is_chuv == 1))

In [11]:
clinical_df[select_id(clinical_df, plc_status=0, missclassified=False, criterion="thickening")].shape

(27, 16)

In [12]:
crit = {
        "thickening": "Peri bronchovascular thickening",
        "pet": "PET_lymphangitis_Visual_analysis",
    }

In [38]:
criterion = "thickening"
print(f"{clinical_df[(~clinical_df[crit[criterion]].isna())].shape[0]} patients with criteria {crit[criterion]}")
print("Among CHUV patients we have the following: ")
print(f"{select_id(clinical_df, plc_status=0, missclassified=False, criterion=criterion).sum()} patients with plc_status=0 and correctly classified")
print(f"{select_id(clinical_df, plc_status=1, missclassified=False, criterion=criterion).sum()} patients with plc_status=1 and correctly classified")
print(f"{select_id(clinical_df, plc_status=0, missclassified=True, criterion=criterion).sum()} patients with plc_status=0 and wrongly classified")
print(f"{select_id(clinical_df, plc_status=1, missclassified=True, criterion=criterion).sum()} patients with plc_status=1 and wrongly classified")

98 patients with criteria Peri bronchovascular thickening
Among CHUV patients we have the following: 
27 patients with plc_status=0 and correctly classified
26 patients with plc_status=1 and correctly classified
2 patients with plc_status=0 and wrongly classified
12 patients with plc_status=1 and wrongly classified


In [14]:
criterion = "thickening"
ids_test = list(
    np.random.choice(
        clinical_df[select_id(
            clinical_df,
            plc_status=0,
            missclassified=True,
            criterion=criterion,
        )].index,
        size=2,
        replace=False,
    ))
print(f"length of ids_test: {len(ids_test)}")
ids_test.extend(
    list(
        np.random.choice(
            clinical_df[select_id(
                clinical_df,
                plc_status=0,
                missclassified=False,
                criterion=criterion,
            )].index,
            size=7,
            replace=False,
        )))
print(f"length of ids_test: {len(ids_test)}")
ids_test.extend(
    list(
        np.random.choice(
            clinical_df[select_id(
                clinical_df,
                plc_status=1,
                missclassified=True,
                criterion=criterion,
            )].index,
            size=12,
            replace=False,
        )))
ids_test.extend(
    list(
        np.random.choice(
            clinical_df[select_id(
                clinical_df,
                plc_status=1,
                missclassified=False,
                criterion=criterion,
            )].index,
            size=7,
            replace=False,
        )))
# ids_test = list(clinical_df[~clinical_df["PET_lymphangitis_Visual_analysis"].isna()].index)

print(f"length of ids_test: {len(set(ids_test))}")
yo = set(ids_test) - set(ids).intersection(ids_test)
ids_test = list(set(ids).intersection(ids_test))
ids_train = list(set(ids) - set(ids_test))
print(f"length of ids_train: {len(ids_train)}")


length of ids_test: 2
length of ids_test: 9
length of ids_test: 28
length of ids_train: 77


In [15]:
df.drop(["plc_status", "voi", "modality"], axis=1)

,log-sigma-1-0-mm-3D_firstorder_Energy,log-sigma-1-0-mm-3D_firstorder_Entropy,log-sigma-1-0-mm-3D_firstorder_Kurtosis,log-sigma-1-0-mm-3D_firstorder_Maximum,log-sigma-1-0-mm-3D_firstorder_Mean,log-sigma-1-0-mm-3D_firstorder_Minimum,log-sigma-1-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation,log-sigma-1-0-mm-3D_firstorder_Skewness,log-sigma-1-0-mm-3D_firstorder_Variance,log-sigma-2-0-mm-3D_firstorder_Energy,...,original_glcm_Imc1,original_glcm_Imc2,original_glcm_InverseVariance,original_glcm_JointAverage,original_glcm_JointEnergy,original_glcm_JointEntropy,original_glcm_MaximumProbability,original_glcm_SumEntropy,original_glcm_SumSquares,patient_id
0,3.273722e+09,3.588292,13.249167,267.106110,5.166340,-585.645142,24.392950,-1.699473,5378.272996,3.322163e+09,...,-0.075355,0.703637,0.172457,16.624791,0.003527,8.776683,0.010055,5.168185,55.666057,PatientLC_51
1,1.388554e+10,4.560236,6.157566,502.399048,-8.928253,-836.504150,56.919238,-0.475070,15229.313807,1.330156e+10,...,-0.138666,0.884338,0.156819,44.756831,0.001420,10.510439,0.005680,6.477741,312.755202,PatientLC_51
2,2.538656e+10,3.858405,12.292960,482.344330,3.532860,-842.676514,29.611399,-1.025939,7572.346312,2.565757e+10,...,-0.041228,0.562417,0.162255,20.286665,0.002414,9.458834,0.006019,5.493089,73.670037,PatientLC_51
3,5.272282e-01,0.919121,3.780066,0.033638,0.004467,-0.050069,0.006620,-0.546626,0.000144,3.268689e+00,...,-0.500475,0.763707,0.092695,1.882100,0.583357,1.336689,0.750356,1.243994,0.196336,PatientLC_51
4,3.133820e+02,1.524041,4.514915,0.175205,-0.080629,-0.749695,0.065489,-1.051585,0.016010,2.885584e+03,...,-0.340513,0.965557,0.448365,14.833176,0.010483,7.002429,0.024624,5.175621,22.733454,PatientLC_51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,1.950340e+10,3.997870,8.186212,768.819092,-8.754198,-502.178894,30.238919,1.151539,13875.814662,1.880986e+10,...,-0.083718,0.752332,0.153207,61.704143,0.002274,9.704139,0.006429,5.711851,215.251230,PatientLC_82
650,2.901077e+10,3.481233,19.089455,295.393738,0.731529,-813.268555,22.039504,-2.799498,5679.855688,2.904775e+10,...,-0.032449,0.492678,0.199672,15.156742,0.004454,8.701366,0.011122,5.059554,58.295469,PatientLC_82
651,1.877236e+00,0.954399,4.561645,0.191203,0.042452,-0.320315,0.045659,-0.970922,0.007631,1.282528e+01,...,-0.430876,0.971079,0.317750,7.105065,0.025276,5.649141,0.053167,4.273600,11.569869,PatientLC_82
652,2.229211e+03,2.515760,3.315819,0.555658,-0.198120,-1.414580,0.148087,-0.366813,0.074640,1.871960e+04,...,-0.233448,0.936786,0.306366,17.581854,0.003598,8.440341,0.006444,5.721697,47.086652,PatientLC_82


In [27]:
features, outcomes = get_formatted_data(df, modality="CT", voi="GTV_T", return_df=True)

In [28]:
X_train = features.loc[ids_train, :].values
X_test = features.loc[ids_test, :].values

y_train = outcomes.loc[ids_train, "plc_status"].values
y_test = outcomes.loc[ids_test, "plc_status"].values

In [29]:
search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('normalization', StandardScaler()),
                                       ('feature_removal',
                                        RemoveHighlyCorrelatedFeatures()),
                                       ('feature_selection', None),
                                       ('classifier', None)]),
             n_jobs=23,
             param_grid=[{'classifier': [RandomForestClassifier()],
                          'classifier__n_estimators': [100, 150],
                          'feature_se...
                          'feature_selection__k': [1, 2, 3, 5]},
                         {'classifier': [LogisticRegression(max_iter=1000,
                                                            penalty='none',
                                                            solver='sag')],
                          'feature_selection': [PCA(n_components=2)],
                   

In [30]:
search.best_estimator_

Pipeline(steps=[('normalization', StandardScaler()),
                ('feature_removal', RemoveHighlyCorrelatedFeatures()),
                ('feature_selection', PCA(n_components=2)),
                ('classifier',
                 LogisticRegression(max_iter=1000, penalty='none',
                                    solver='sag'))])

In [31]:
[k for k in features.columns if "ori" in k]

['original_firstorder_Energy',
 'original_firstorder_Entropy',
 'original_firstorder_Kurtosis',
 'original_firstorder_Maximum',
 'original_firstorder_Mean',
 'original_firstorder_Minimum',
 'original_firstorder_RobustMeanAbsoluteDeviation',
 'original_firstorder_Skewness',
 'original_firstorder_Variance',
 'original_glcm_Autocorrelation',
 'original_glcm_ClusterProminence',
 'original_glcm_ClusterShade',
 'original_glcm_ClusterTendency',
 'original_glcm_Contrast',
 'original_glcm_Correlation',
 'original_glcm_DifferenceAverage',
 'original_glcm_DifferenceEntropy',
 'original_glcm_DifferenceVariance',
 'original_glcm_Id',
 'original_glcm_Idm',
 'original_glcm_Idmn',
 'original_glcm_Idn',
 'original_glcm_Imc1',
 'original_glcm_Imc2',
 'original_glcm_InverseVariance',
 'original_glcm_JointAverage',
 'original_glcm_JointEnergy',
 'original_glcm_JointEntropy',
 'original_glcm_MaximumProbability',
 'original_glcm_SumEntropy',
 'original_glcm_SumSquares']

In [32]:
y_pred_test = search.predict(X_test)
suv_test = features.loc[ids_test, "original_firstorder_Mean"].values
y_pred_test_control = clinical_df.loc[ids_test, "Peri bronchovascular thickening"].values

In [33]:
y_pred_test

array([1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.])

In [34]:
np.exp(delong_roc_test(y_test, suv_test, y_pred_test_control))

array([[0.95640245]])

In [35]:
alpha = .95


auc, auc_cov = delong_roc_variance(
    y_test,
    y_pred_test)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('AUC:', auc)
print('AUC COV:', auc_cov)
print('95% AUC CI:', ci)

AUC: 0.5087719298245614
AUC COV: 0.009945367805478608
95% AUC CI: [0.31331165 0.70423221]


In [36]:
alpha = .95


auc, auc_cov = delong_roc_variance(
    y_test,
    y_pred_test_control)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('AUC:', auc)
print('AUC COV:', auc_cov)
print('95% AUC CI:', ci)

AUC: 0.5730994152046782
AUC COV: 0.008632998187476489
95% AUC CI: [0.39099146 0.75520737]


In [37]:
alpha = .95

auc, auc_cov = delong_roc_variance(
    y_test,
    suv_test)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('AUC:', auc)
print('AUC COV:', auc_cov)
print('95% AUC CI:', ci)

AUC: 0.5906432748538011
AUC COV: 0.0177974989455445
95% AUC CI: [0.32916984 0.85211671]
